# PySR

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(f"data/6000_points.csv")
if 'system:index' in df.columns:
    df.drop('system:index', axis=1, inplace=True)
    
if '.geo' in df.columns:
    df.drop('.geo', axis=1, inplace=True)
df['B8'] = (df['B8'] - df[['B8', 'B4']].min().min())/(df[['B8', 'B4']].max().max()-df[['B8', 'B4']].min().min())
df['B4'] = (df['B4'] - df[['B8', 'B4']].min().min())/(df[['B8', 'B4']].max().max()-df[['B8', 'B4']].min().min())
df['NDVI'] = (df['B8']-df['B4'])/(df['B8']+df['B4'])
df.to_csv('data/ndvi.csv')

In [2]:
df.head()

,B1,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,QA60,date,lat,lon,NDVI
0,1380,3202,2266,1245,1101,0.110992,1413,1716,1944,0.157315,2250,0,2023-03-23,37.335014,-95.326731,0.172649
1,1317,49,42,889,503,0.025779,247,208,190,0.004375,146,0,2023-06-04,-21.011541,-124.765524,-0.709847
2,1162,2031,1475,951,850,0.086645,1036,1136,1265,0.092202,1298,0,2023-01-26,11.062935,-11.792313,0.031068
3,1286,1291,444,980,823,0.047619,832,1995,2736,0.201565,3134,0,2023-10-24,-1.910390,114.546423,0.617800
4,1381,2050,1363,1126,1016,0.087272,1155,1617,1845,0.144444,2015,0,2023-04-13,27.868199,120.743335,0.246733


In [3]:
x = df.drop(['NDVI', 'date'], axis=1)
y = df['NDVI']

# DRAGON: Symbolic Regression

In [4]:
import torch
import torch.nn as nn
from dragon.search_space.dag_encoding import AdjMatrix
import os

class MetaArchi(nn.Module):
    def __init__(self, args, input_shape):
        super().__init__()
        self.input_shape = input_shape
        assert isinstance(args['Dag'], AdjMatrix), f"The 'Dag' argument should be an 'AdjMatrix'. Got {type(args['Dag'])} instead."
        self.dag = args['Dag']
        self.dag.set(input_shape)


    def forward(self, X):
        return self.dag(X)
    def set_prediction_to_save(self, name, df):
        if hasattr(self, "prediction"):
            self.prediction[name] = df
        else:
            self.prediction = {name: df}
    def save(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        full_path = os.path.join(path, "best_model.pth")
        torch.save(self.state_dict(), full_path)
        if hasattr(self, "prediction"):
            for k in self.prediction.keys():
                self.prediction[k].to_csv(os.path.join(path, f"best_model_{k}_outputs.csv"))

In [5]:
from torch.utils.data import Dataset, DataLoader

class RegressionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X.values)
        self.y = torch.FloatTensor(y.values)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
X = df[['B8', 'B4']]
z = df[['NDVI']]
train_set = RegressionDataset(X, z)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

In [6]:
from dragon.search_operators.base_neighborhoods import CatInterval, ConstantInterval
from dragon.search_operators.dag_neighborhoods import EvoDagInterval, HpInterval
from dragon.search_space.base_variables import CatVar, Constant
from dragon.search_space.bricks.basics import Identity
from dragon.search_space.bricks.symbolic_regression import *
from dragon.search_space.dag_variables import EvoDagVariable, HpVar
from dragon.search_space.bricks_variables import (
    operations_var,
    dag_var,
)
from dragon.search_space.base_variables import ArrayVar
from dragon.search_space.dag_encoding import SymbolicNode
from dragon.search_operators.base_neighborhoods import ArrayInterval


unary_var = HpVar(
    "UnaryOp",
    CatVar(
        "UnaryOpType",
        features=[Identity, Inverse, Negate],
        neighbor=CatInterval()
    ),
    hyperparameters={},
    neighbor=HpInterval()
)
select_features_var = HpVar(
    "SelectFeatures",
    Constant("SelectFeaturesOp", SelectFeatures, neighbor=ConstantInterval()),
    hyperparameters={
        "feature_indices": CatVar(
            "feature_indices",
            features=[[0], [1]],
            neighbor=CatInterval()
        )
    },
    neighbor=HpInterval()
)

sum_var = HpVar(
    "Sum",
    Constant("SumOp", SumFeatures, neighbor=ConstantInterval()),
    hyperparameters={},
    neighbor=HpInterval()
)


candidate_operations = operations_var(
    "CandidateOperations",
    size=10,
    candidates=[
        select_features_var,
        unary_var,
        sum_var
    ],
    activations=Constant(
        "id",
        value=nn.Identity(),
        neighbor=ConstantInterval()
    ),
    node_type=SymbolicNode
)

dag = EvoDagVariable(
                    label="Dag",
                    operations = candidate_operations,
                    init_complexity=8,
                    neighbor=EvoDagInterval(nb_mutations=1) 
                )


search_space = ArrayVar(dag, label="Search Space", neighbor=ArrayInterval())

In [7]:
from dragon.utils.plot_functions import expr_to_mini_dag, graph_to_formula

import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from dragon.utils.plot_functions import graph_to_formula

def eval(args, eps=1e-1,):
    model = MetaArchi(args, input_shape=(2,)).to(device)
    model.eval()

    all_pred, all_true = [], []

    with torch.no_grad():
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            pred = model(Xb)
            all_pred.append(pred.detach().cpu())
            all_true.append(yb.detach().cpu())
    pred_all = torch.cat(all_pred)
    true_all = torch.cat(all_true)

    mse = np.inf
    selected_c = 0
    if len(pred_all.shape) > 1:
        for c in range(pred_all.shape[-1]):
            loss = nn.MSELoss()(pred_all[:, c:c+1], true_all)
            if loss < mse:
                mse = loss
                selected_c = c
        pred_all = pred_all[:, selected_c]

    pred_all = pred_all.squeeze().numpy()
    true_all = true_all.squeeze().numpy()

    close_matches = np.sum(np.abs(pred_all - true_all) < eps)
    fraction_close = close_matches / len(pred_all)
    var = np.abs(true_all.std() - pred_all.std())
    return model, pred_all, true_all, mse, fraction_close, var
    

def loss_function(args, idx, eps=1e-1, mse_threshold=0.04, *kwargs):
    labels = [e.label for e in search_space]
    args = dict(zip(labels, args))

    model, y_pred, y_true, mse, fraction_close, var = eval(args, eps)

    try:
        expr = graph_to_formula(
            args['Dag'].matrix,
            np.asarray(['B4', 'B8']),
            args['Dag'].operations
        )
    except Exception as e:
        expr = None

    print(
        f"Idx={idx}, MSE = {mse:.6f}, var={var:.6f} "
        f"fraction_close = {fraction_close:.4f}, "
        f"formula={expr}"
    )
    rand = np.random.rand()
    if rand > 0.5:
        if expr is not None:
            try:
                mini_dag = expr_to_mini_dag(expr, ["B4", "B8"])
                args['Dag'] = mini_dag
                model, y_pred, y_true, mse, fraction_close, var = eval(args, eps)
                expr = graph_to_formula(
                    args['Dag'].matrix,
                    np.asarray(['B4', 'B8']),
                    args['Dag'].operations
                )
                print(f"Idx={idx}, mini dag MSE={mse:.6f}, var = {var:.6f}, formula = {expr}")
            except Exception as e:
                pass
        model.mini_x = [v for v in args.values()]
    df = pd.DataFrame({
        "pred": y_pred,
        "true": y_true
    })
    model.set_prediction_to_save("prediction", df)
    return (mse+var).item(), model


In [8]:
from dragon.search_space.bricks.basics import Identity
from dragon.search_space.dag_encoding import SymbolicNode
from dragon.search_space.bricks.symbolic_regression import *


m_dag = [SymbolicNode(combiner='add', operation=Identity, hp = {}, activation=nn.Identity()), 
         SymbolicNode(combiner='add', operation=SumFeatures, hp = {}, activation=nn.Identity()), 
         SymbolicNode(combiner='mul', operation=Negate, hp={}, activation=nn.Identity()), 
         SymbolicNode(combiner='add', operation=SelectFeatures, hp={'feature_indices': [0]}, activation=nn.Identity()), 
         SymbolicNode(combiner='add', operation=SelectFeatures, hp={'feature_indices': [1]}, activation=nn.Identity()), 
         SymbolicNode(combiner='add', operation=Inverse, hp={}, activation=nn.Identity()), 
         SymbolicNode(combiner='mul', operation=Identity, hp={}, activation=nn.Identity()),
         SymbolicNode(combiner='add', operation=Inverse, hp={}, activation=nn.Identity())]
n_dag = np.array([[0, 1, 1, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 0, 0, 0]
                  ])

minimal_graph = [AdjMatrix(matrix=n_dag, operations=m_dag)]
loss, model = loss_function(minimal_graph, idx="my optimal graph")
print("Initial MSE:", loss)
#loss, model = loss_function(model.mini_x, idx="my minimal optimal graph")


from sympy import symbols, sympify

B4, B8 = symbols("B4 B8")

tests = [
    B4 / B8,
    (B4 + B8) / B4,
    -(B4 * B8),
    (B4 - B8) / (B8 + B4),
    -B4,
    sympify(0),
    sympify(-1/3),
    B4**3,
    B4**2+B8**2
]

for expr in tests:
    print("\nExpression: ", expr)
    dag = expr_to_mini_dag(expr, ["B4", "B8"])
    loss, model = loss_function([dag], 'test')

Idx=my optimal graph, MSE = 0.000000, var=0.000000 fraction_close = 1.0000, formula=(B4 - B8)/(B4 + B8)
Idx=my optimal graph, mini dag MSE=0.000000, var = 0.000000, formula = (B4 - B8)/(B4 + B8)
Initial MSE: 2.980232949312267e-08

Expression:  B4/B8
Idx=test, MSE = 3.198398, var=1.037242 fraction_close = 0.0000, formula=B4/B8

Expression:  (B4 + B8)/B4
Idx=test, MSE = 1480071424.000000, var=38463.917969 fraction_close = 0.0000, formula=(B4 + B8)/B4

Expression:  -B4*B8
Idx=test, MSE = 0.153717, var=0.233827 fraction_close = 0.1645, formula=-B4*B8

Expression:  (B4 - B8)/(B4 + B8)
Idx=test, MSE = 0.000000, var=0.000000 fraction_close = 1.0000, formula=(B4 - B8)/(B4 + B8)
Idx=test, mini dag MSE=0.000000, var = 0.000000, formula = (B4 - B8)/(B4 + B8)

Expression:  -B4
Idx=test, MSE = 0.241111, var=0.186447 fraction_close = 0.0634, formula=-B4

Expression:  0
Idx=test, MSE = 0.133006, var=0.364129 fraction_close = 0.3359, formula=0.0
Idx=test, mini dag MSE=0.133006, var = 0.364129, formula

In [9]:
from dragon.search_algorithm.agentic.generic_multi_agent_integration import create_generic_multi_agent_search
from dragon.search_algorithm.mutant_ucb import Mutant_UCB

T_mut = 500
K = int(0.3*T_mut)
save_dir="save/NDVI_formula/"
search_algorithm = Mutant_UCB(
    search_space=search_space,
    evaluation=loss_function,
    T=T_mut,
    K=K,
    N=1,
    E=100,
    save_dir=save_dir,
    clean_all=False,
    verbose=True
)
search_algorithm.run()
T_mut+=500
search_algorithm = Mutant_UCB(
    search_space=search_space,
    evaluation=loss_function,
    T=T_mut,
    K=K,
    N=1,
    E=0.01,
    save_dir=save_dir,
    clean_all=False,
    verbose=True,
    pop_path=save_dir
)
search_algorithm.run()

2026-02-03 10:24:34,884 | WARNING | Use multiple processes if you want to use the distributed version.
2026-02-03 10:24:34,887 | INFO | save/NDVI_formula/ already exists. Deleting it.
2026-02-03 10:24:35,271 | INFO | The whole population has been created (size = 150), 150 have been randomy initialized.
Idx=0, MSE = 0.133006, var=0.364129 fraction_close = 0.3359, formula=0
2026-02-03 10:24:35,368 | INFO | Best found! 0.4971356987953186 < inf
Idx=1, MSE = 0.241112, var=0.186447 fraction_close = 0.0634, formula=-B4
Idx=1, mini dag MSE=0.241111, var = 0.186447, formula = -B4
2026-02-03 10:24:35,503 | INFO | Best found! 0.4275585114955902 < 0.4971356987953186
Idx=2, MSE = 3816794095616.000000, var=1953287.375000 fraction_close = 0.0000, formula=1/B4
Idx=3, MSE = 3816797765632.000000, var=1953288.375000 fraction_close = 0.0000, formula=(2*B4 + B8)/(B4*(B4 + B8))
Idx=3, mini dag MSE=3816790425600.000000, var = 1953286.375000, formula = (2.0*B4 + B8)/(B4*(B4 + B8))
Idx=4, MSE = 0.159621, var=0

In [10]:
from dragon.utils.plot_functions import load_archi, graph_to_formula

best_model = load_archi(f"{save_dir}/best_model/x.pkl")
labels = [e.label for e in search_space]
best_model = dict(zip(labels, best_model))
expr = graph_to_formula(best_model['Dag'].matrix, np.asarray(['B4', 'B8']), best_model['Dag'].operations)
expr

-1/B4

In [ ]:
from dragon.search_algorithm.agentic.generic_multi_agent_integration import create_generic_multi_agent_search
from dragon.search_algorithm.agentic.symbolic_formula_analysis import create_symbolic_guided_system
from dragon.search_algorithm.agentic.compact_prompt_fix import create_compact_agents

api_key=""
save_dir="save/NDVI_formula/"

algo = create_generic_multi_agent_search(
    search_space=search_space,
    save_dir=save_dir,
    api_key=api_key,
    task_description="symbolic regression on 2 features",
    evaluation=loss_function,
    model="meta-llama/llama-guard-4-12b",
    T=T_mut+20, K=10, N=1,
    pop_path=save_dir
)

algo.agent_policy = create_symbolic_guided_system(
    base_policy=algo.agent_policy,
    graph_to_formula_fn=graph_to_formula,
    expr_to_mini_dag_fn=expr_to_mini_dag,
    input_names=['B4', 'B8']
)

algo.agent_policy = create_compact_agents(algo.agent_policy, api_key=api_key, task_desc="symbolic regression on 2 features")

algo.run()

2026-02-03 10:31:03,671 | INFO | Extracting complete search space...
2026-02-03 10:31:03,681 | INFO | Extracted 5 operations: ['Identity', 'Inverse', 'Negate', 'SelectFeatures', 'SumFeatures']
2026-02-03 10:31:03,683 | INFO | Extracted 3 combiners: ['add', 'concat', 'mul']
2026-02-03 10:31:03,684 | INFO | Extracted 1 activations: ['Identity']
2026-02-03 10:31:03,958 | INFO | Generic multi-agent policy initialized
2026-02-03 10:31:03,960 | INFO |   Operations: 5
2026-02-03 10:31:03,961 | INFO |   Combiners: 3
2026-02-03 10:31:03,961 | INFO |   Activations: 1
2026-02-03 10:31:03,962 | INFO | Extracting complete search space...
2026-02-03 10:31:03,967 | INFO | Extracted 5 operations: ['Identity', 'Inverse', 'Negate', 'SelectFeatures', 'SumFeatures']
2026-02-03 10:31:03,969 | INFO | Extracted 3 combiners: ['add', 'concat', 'mul']
2026-02-03 10:31:03,969 | INFO | Extracted 1 activations: ['Identity']
2026-02-03 10:31:04,109 | WARNING | Use multiple processes if you want to use the distribut

In [ ]:
from dragon.utils.plot_functions import load_archi, graph_to_formula

best_model = load_archi(f"{save_dir}/best_model/x.pkl")
labels = [e.label for e in search_space]
best_model = dict(zip(labels, best_model))
expr = graph_to_formula(best_model['Dag'].matrix, np.asarray(['B4', 'B8']), best_model['Dag'].operations)
expr